# 性能テスト結果分析

このノートブックは `performance_test.ipynb` で実行された性能テストの結果を分析し、可視化します。

## 分析内容
1. **データの読み込みと前処理**: `analysis/data` から最新のテスト結果を読み込みます。
2. **全体サマリー**: 全テスト結果の概要を表示します。
3. **サービス間性能比較**: レスポンス時間やスループット(QPS)をサービス間で比較します。
4. **負荷パターン別分析**: 負荷（軽・中・高）による性能の変化を分析します。
5. **スケーラビリティ分析**: 負荷増加に対する各サービスの性能維持能力を評価します。
6. **リソース使用量分析**: コンテナのCPU・メモリ使用量を分析します。
7. **レポート生成**: 分析結果をまとめたMarkdownレポートを生成し、`analysis/report` に保存します。

In [1]:
from pathlib import Path
import glob
import os
import json
from datetime import datetime
import pandas as pd
import numpy as np
from IPython.display import display

# 全体で使うライブラリをまとめてインポートします

import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
# パス設定
BASE_DIR = Path.cwd().parent.parent
DATA_DIR = BASE_DIR / "analysis" / "data"
REPORT_DIR = BASE_DIR / "analysis" / "report"
REPORT_DIR.mkdir(parents=True, exist_ok=True)

## 1. データの読み込みと前処理

In [ ]:
def load_latest_test_data(data_dir):
    """最新の包括的テスト結果(JSON)を読み込む"""
    list_of_files = glob.glob(str(data_dir / 'comprehensive_test_*.json'))
    if not list_of_files:
        print("❌ テスト結果ファイルが見つかりません。")
        return None

    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"📂 最新のテスト結果ファイルを読み込み中: {os.path.basename(latest_file)}")

    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # エラーを含む結果を除外してDataFrameに変換
    valid_data = [d for d in data if 'error' not in d]
    df = pd.DataFrame(valid_data)

    if df.empty:
        print("⚠️ 有効なテストデータがありません。")
        return None

    # データ型の整理
    numeric_cols = [
        'clients', 'total_requests', 'successful_requests', 'duration',
        'error_rate', 'qps', 'avg_response_time', 'median_response_time',
        'p95_response_time', 'p99_response_time', 'min_response_time', 'max_response_time'
    ]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 負荷パターンの順序を定義
    df['pattern_name'] = pd.Categorical(df['pattern_name'], categories=['light', 'medium', 'heavy'], ordered=True)

    print(f"✅ {len(df)}件の有効なテスト結果を読み込みました。")
    return df

df_results = load_latest_test_data(DATA_DIR)

if df_results is not None:
    df_results.head()

📂 最新のテスト結果ファイルを読み込み中: comprehensive_test_20250912_131633.json
✅ 30件の有効なテスト結果を読み込みました。


## 2. 全体サマリー

In [ ]:
if df_results is not None:
    # サマリーテーブルの表示
    summary = df_results.pivot_table(
        index='service_name',
        columns='pattern_name',
        values=['qps', 'avg_response_time', 'error_rate'],
        aggfunc='mean'
    ).fillna(0)

    # カラムの整形
    summary.columns = [f'{val}_{cat}' for val, cat in summary.columns]

    # 表示形式の調整
    summary_styled = summary.style.format({
        col: '{:.2f}' for col in summary.columns if 'qps' in col or 'avg_response_time' in col
    }).format({
        col: '{:.2f}%' for col in summary.columns if 'error_rate' in col
    }).set_caption("📊 全体パフォーマンスサマリー")

    display(summary_styled)
else:
    print("データがありません。")

/var/folders/nc/w7rlvnrs361c069dkdz3nwnw0000gn/T/ipykernel_8579/1001240079.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = df_results.pivot_table(


,avg_response_time_light,avg_response_time_medium,avg_response_time_heavy,error_rate_light,error_rate_medium,error_rate_heavy,qps_light,qps_medium,qps_heavy
service_name,,,,,,,,,
1-coordinator-ring,3.301427,3.985969,4.009650,0.00%,0.00%,0.00%,300.154195,248.468175,247.046580
10-leaderboard,3.875364,3.866973,4.038517,0.00%,0.00%,0.00%,255.622479,256.091440,245.261303
2-quorum-consistency,5.893178,5.156218,5.694512,0.00%,0.00%,0.00%,168.263685,192.173713,174.031692
3-sharding-replica,4.307514,4.665132,4.792379,0.00%,0.00%,0.00%,230.173694,212.480700,206.864975
4-distributed-lock,5.085132,4.204673,4.460276,0.00%,0.00%,0.00%,194.986097,235.584241,222.096130
5-cache-aside,7.886355,7.976751,7.534616,0.00%,0.00%,0.00%,125.640879,124.206106,131.545441
6-bloom-sstable,3.971055,3.920259,3.987017,0.00%,0.00%,99.60%,249.471443,252.654482,0.891575
7-rate-limiting,3.731406,3.691007,3.794234,0.00%,0.00%,0.00%,265.594235,268.429821,261.160199
8-line-streams,3.777341,3.848923,3.978330,0.00%,0.00%,0.00%,262.378271,257.424264,249.063163


## 3. サービス間性能比較

In [5]:
if df_results is not None:
    fig = px.bar(
        df_results,
        x='service_name',
        y='qps',
        color='pattern_name',
        barmode='group',
        title='🚀 サービス別スループット(QPS)比較',
        labels={'service_name': 'サービス名', 'qps': 'QPS (Queries Per Second)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="QPS (高いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [6]:
if df_results is not None:
    fig = px.bar(
        df_results,
        x='service_name',
        y='avg_response_time',
        color='pattern_name',
        barmode='group',
        title='⏱️ サービス別平均レスポンス時間比較',
        labels={'service_name': 'サービス名', 'avg_response_time': '平均レスポンス時間 (ms)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="平均レスポンス時間 (ms, 低いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [ ]:
if df_results is not None:
    df_melted = df_results.melt(
        id_vars=['service_name', 'pattern_name'],
        value_vars=['p95_response_time', 'p99_response_time'],
        var_name='percentile',
        value_name='response_time'
    )

    fig = px.bar(
        df_melted,
        x='service_name',
        y='response_time',
        color='percentile',
        facet_col='pattern_name',
        barmode='group',
        title='レスポンス時間パーセンタイル比較 (95%tile, 99%tile)',
        labels={'service_name': 'サービス名', 'response_time': 'レスポンス時間 (ms)', 'percentile': 'パーセンタイル'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_yaxes(matches=None)
    fig.show()
else:
    print("データがありません。")

## 4. スケーラビリティ分析

In [8]:
if df_results is not None:
    fig = px.line(
        df_results.sort_values('pattern_name'),
        x='pattern_name',
        y='qps',
        color='service_name',
        markers=True,
        title='📈 負荷増加に伴うQPSの変化 (スケーラビリティ)',
        labels={'pattern_name': '負荷パターン', 'qps': 'QPS', 'service_name': 'サービス名'}
    )
    fig.update_layout(xaxis_title="負荷パターン", yaxis_title="QPS (高いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [9]:
if df_results is not None:
    fig = px.line(
        df_results.sort_values('pattern_name'),
        x='pattern_name',
        y='avg_response_time',
        color='service_name',
        markers=True,
        title='📉 負荷増加に伴う平均レスポンス時間の変化',
        labels={'pattern_name': '負荷パターン', 'avg_response_time': '平均レスポンス時間 (ms)', 'service_name': 'サービス名'}
    )
    fig.update_layout(xaxis_title="負荷パターン", yaxis_title="平均レスポンス時間 (ms, 低いほど良い)")
    fig.show()
else:
    print("データがありません。")

## 5. リソース使用量分析

注意: この分析は `performance_test.ipynb` でリソース情報が正しく収集されている場合にのみ有効です。

In [10]:
def parse_resource_stats(df):
    """リソース統計情報をパースしてDataFrameに追加

    対応フォーマット:
    - 元の: 'final_stats' 列が辞書 (container_name -> {cpu_percent, memory_usage, ...})
    - CSV向けフラット化: 'final_cpu_avg', 'final_mem_usage_avg' などの数値列
    """
    # 元の辞書形式があれば従来処理を優先
    if 'final_stats' in df.columns:
        cpu_usage = []
        mem_usage = []

        for index, row in df.iterrows():
            stats = row['final_stats']
            if isinstance(stats, dict) and stats:
                # 'app'コンテナの統計を優先
                app_container_key = next((k for k in stats.keys() if 'app' in k), None)
                if app_container_key:
                    cpu_usage.append(stats[app_container_key].get('cpu_percent', np.nan))
                    mem_usage.append(stats[app_container_key].get('memory_usage', np.nan) / (1024*1024)) # MBに変換
                else: # 'app'が見つからない場合は最初のコンテナ
                    first_key = list(stats.keys())[0]
                    cpu_usage.append(stats[first_key].get('cpu_percent', np.nan))
                    mem_usage.append(stats[first_key].get('memory_usage', np.nan) / (1024*1024)) # MBに変換
            else:
                cpu_usage.append(np.nan)
                mem_usage.append(np.nan)

        df['cpu_percent'] = cpu_usage
        df['memory_usage_mb'] = mem_usage
        return df

    # CSVフラット化列に対応
    # 期待する列例: final_cpu_avg, final_cpu_max, final_mem_usage_avg, final_mem_usage_max
    cpu_col_candidates = ['final_cpu_avg', 'final_cpu_max', 'initial_cpu_avg', 'initial_cpu_max']
    mem_col_candidates = ['final_mem_usage_avg', 'final_mem_usage_max', 'initial_mem_usage_avg', 'initial_mem_usage_max']

    cpu_col = next((c for c in cpu_col_candidates if c in df.columns), None)
    mem_col = next((c for c in mem_col_candidates if c in df.columns), None)

    if cpu_col or mem_col:
        # cpu はパーセントのまま、mem はバイト単位であればMBに変換する
        df['cpu_percent'] = df[cpu_col] if cpu_col in df.columns else np.nan
        if mem_col in df.columns:
            # もしすでにMBに変換済みの列名であればそのまま使う。ここでは元がバイトとして扱う。
            df['memory_usage_mb'] = df[mem_col].astype(float) / (1024*1024)
        else:
            df['memory_usage_mb'] = np.nan

        return df

    print("⚠️ リソース統計情報 (final_stats または final_* 列) が見つかりません。")
    return df


if df_results is not None:
    df_resources = parse_resource_stats(df_results.copy())
    if 'cpu_percent' in df_resources.columns:
        display(df_resources[['service_name', 'pattern_name', 'cpu_percent', 'memory_usage_mb']].head())
else:
    print("データがありません。")

,service_name,pattern_name,cpu_percent,memory_usage_mb
0,1-coordinator-ring,light,0.018805,77.167969
1,1-coordinator-ring,medium,0.041834,69.281250
2,1-coordinator-ring,heavy,0.020944,65.781250
3,2-quorum-consistency,light,0.029133,70.585938
4,2-quorum-consistency,medium,0.036830,87.273438


In [11]:
if df_results is not None and 'cpu_percent' in df_resources.columns:
    fig = px.bar(
        df_resources.dropna(subset=['cpu_percent']),
        x='service_name',
        y='cpu_percent',
        color='pattern_name',
        barmode='group',
        title='💻 サービス別CPU使用率',
        labels={'service_name': 'サービス名', 'cpu_percent': 'CPU使用率 (%)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="CPU使用率 (%)")
    fig.show()
else:
    print("CPUデータがありません。")

In [12]:
if df_results is not None and 'memory_usage_mb' in df_resources.columns:
    fig = px.bar(
        df_resources.dropna(subset=['memory_usage_mb']),
        x='service_name',
        y='memory_usage_mb',
        color='pattern_name',
        barmode='group',
        title='🧠 サービス別メモリ使用量',
        labels={'service_name': 'サービス名', 'memory_usage_mb': 'メモリ使用量 (MB)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="メモリ使用量 (MB)")
    fig.show()
else:
    print("メモリデータがありません。")

## 6. レポート生成

In [ ]:
def generate_report(df):
    """Markdown形式のレポートを生成（堅牢化）"""
    import math
    if df is None or df.empty:
        return "# 性能テストレポート\n\nテストデータが見つかりませんでした。"

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    report = f"# 性能テスト分析レポート\n\n生成日時: {timestamp}\n\n"

    report += "## 1. 総合評価\n\n"

    # 高負荷時のQPSランキング（存在すれば）
    heavy_load_df = df[df['pattern_name'] == 'heavy'].sort_values('qps', ascending=False)
    if not heavy_load_df.empty:
        report += "### 高負荷時 QPSランキング\n"
        for _, row in heavy_load_df.iterrows():
            qps = row.get('qps', None)
            qps_str = f"{qps:.2f} QPS" if (qps is not None and not math.isnan(qps)) else "N/A"
            report += f"- **{row['service_name']}**: {qps_str}\n"
        report += "\n"

        report += "### 高負荷時 平均レスポンス時間ランキング (短い順)\n"
        for _, row in heavy_load_df.sort_values('avg_response_time').iterrows():
            art = row.get('avg_response_time', None)
            art_str = f"{art:.2f} ms" if (art is not None and not math.isnan(art)) else "N/A"
            report += f"- **{row['service_name']}**: {art_str}\n"
        report += "\n"
    else:
        report += "⚠️ 高負荷時のデータが見つかりませんでした。\n\n"

    report += "## 2. サマリーテーブル\n\n"
    try:
        summary_df = df.pivot_table(
            index='service_name',
            columns='pattern_name',
            values=['qps', 'avg_response_time']
        ).fillna(0)
        report += summary_df.to_markdown(floatfmt=".2f")
    except Exception as e:
        report += f"サマリーテーブルを作成できませんでした: {e}\n"
    report += "\n\n"

    report += "## 3. 設計パターン別特性分析\n\n"
    for service_name in sorted(df['service_name'].unique()):
        report += f"### {service_name}\n"
        service_df = df[df['service_name'] == service_name]

        # 安全にlight/heavy行を取得する（存在しない場合は代替を使う）
        light_df = service_df[service_df['pattern_name'] == 'light']
        heavy_df = service_df[service_df['pattern_name'] == 'heavy']

        if not light_df.empty:
            light = light_df.iloc[0]
        elif not service_df.empty:
            # lightが無ければclientsが最小の行を代わりに使う
            light = service_df.sort_values('clients').iloc[0]
        else:
            report += "データが不足しています。\n\n"
            continue

        if not heavy_df.empty:
            heavy = heavy_df.iloc[0]
        elif not service_df.empty:
            # heavyが無ければclientsが最大の行を代わりに使う
            heavy = service_df.sort_values('clients', ascending=False).iloc[0]
        else:
            report += "データが不足しています。\n\n"
            continue

        # 安全な数値取得
        def safe_num(val):
            try:
                return float(val)
            except Exception:
                return float('nan')

        light_qps = safe_num(light.get('qps', float('nan')))
        heavy_qps = safe_num(heavy.get('qps', float('nan')))
        light_resp = safe_num(light.get('avg_response_time', float('nan')))
        heavy_resp = safe_num(heavy.get('avg_response_time', float('nan')))

        qps_growth = heavy_qps / light_qps if (light_qps and not math.isnan(light_qps)) else float('nan')
        resp_growth = heavy_resp / light_resp if (light_resp and not math.isnan(light_resp)) else float('nan')

        qps_line = f"軽負荷時 {light_qps:.2f} QPS → 高負荷時 {heavy_qps:.2f} QPS ({qps_growth:.2f}倍)" if (not math.isnan(light_qps) and not math.isnan(heavy_qps) and not math.isnan(qps_growth)) else "スループットデータ不足"
        resp_line = f"軽負荷時 {light_resp:.2f} ms → 高負荷時 {heavy_resp:.2f} ms ({resp_growth:.2f}倍)" if (not math.isnan(light_resp) and not math.isnan(heavy_resp) and not math.isnan(resp_growth)) else "レスポンス時間データ不足"

        report += f"- **スループット**: {qps_line}\n"
        report += f"- **レスポンス時間**: {resp_line}\n"

        # 考察
        if not math.isnan(resp_growth) and resp_growth > 5:
            report += f"- **考察**: ⚠️ 高負荷時にレスポンス時間が大幅に悪化しています。ボトルネックの可能性があります。\n"
        elif not math.isnan(qps_growth) and qps_growth < 2 and (light.get('clients', 0) and heavy.get('clients', 0) and heavy.get('clients', 0)/light.get('clients', 1) > 5):
            report += f"- **考察**: ⚠️ 負荷を増やしてもスループットが伸び悩んでいます。スケーラビリティに課題があるかもしれません。\n"
        else:
            report += f"- **考察**: ✅ 負荷に対して比較的安定した性能を維持しています。\n"

        report += "\n"

    return report


# レポート生成と保存処理
if df_results is not None:
    report_content = generate_report(df_resources if 'df_resources' in globals() else df_results)

    # レポートをファイルに保存
    report_filename = f"performance_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    report_path = REPORT_DIR / report_filename
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_content)

    # レポートのプレビュー
    print("\n--- レポートプレビュー ---")
    print(report_content[:1000] + "...")
else:
    print("データがないためレポートを生成できません。")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

# Font settings (DejaVu Sans is a good default for wide character support)
# plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10

# Data definition
patterns = [
    'coordinator-ring',
    'quorum-consistency',
    'sharding-replica',
    'distributed-lock',
    'cache-aside',
    'bloom-sstable',
    'rate-limiting',
    'line-streams',
    'session-store',
    'leaderboard'
]

# English labels
pattern_labels = [
    '1. coordinator-ring',
    '2. quorum-consistency',
    '3. sharding-replica',
    '4. distributed-lock',
    '5. cache-aside',
    '6. bloom-sstable',
    '7. rate-limiting',
    '8. line-streams',
    '9. session-store',
    '10. leaderboard'
]

metrics = [
    'QPS\n(Light Load)',
    'QPS\n(Medium Load)',
    'QPS\n(Heavy Load)',
    'Response\nTime',
    'CPU\nEfficiency',
    'Stability',
    'Overall\nRating'
]

# Performance scores (1-5, 5 is the best)
scores = np.array([
    [5, 4, 4, 4, 4, 3, 4],  # coordinator-ring
    [2, 3, 2, 2, 4, 4, 3],  # quorum-consistency
    [4, 3, 3, 4, 4, 4, 4],  # sharding-replica
    [3, 4, 4, 4, 1, 3, 2],  # distributed-lock
    [2, 2, 2, 1, 5, 5, 5],  # cache-aside
    [4, 4, 1, 5, 4, 1, 1],  # bloom-sstable
    [5, 5, 5, 5, 5, 5, 5],  # rate-limiting
    [5, 4, 5, 5, 4, 4, 5],  # line-streams
    [4, 4, 4, 4, 4, 4, 4],  # session-store
    [4, 4, 4, 5, 4, 5, 5]   # leaderboard
])

def create_heatmap():
    """Creates the main heatmap."""
    fig, ax = plt.subplots(figsize=(12, 10))

    # Create custom colormap (Red -> Orange -> Green)
    colors = ['#e74c3c', '#e67e22', '#f39c12', '#2ecc71', '#27ae60']
    n_bins = 100
    cmap = LinearSegmentedColormap.from_list('custom', colors, N=n_bins)

    # Create heatmap
    im = ax.imshow(scores, cmap=cmap, aspect='auto', vmin=1, vmax=5)

    # Axis settings
    ax.set_xticks(np.arange(len(metrics)))
    ax.set_yticks(np.arange(len(patterns)))
    ax.set_xticklabels(metrics, rotation=0, ha='center')
    ax.set_yticklabels(pattern_labels, ha='right')

    # Grid lines
    ax.set_xticks(np.arange(len(metrics)+1)-.5, minor=True)
    ax.set_yticks(np.arange(len(patterns)+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=2)

    # Display values in each cell
    for i in range(len(patterns)):
        for j in range(len(metrics)):
            score = scores[i, j]
            # Labels based on score
            labels = ['Poor', 'Low', 'Average', 'Good', 'Excellent']
            label = labels[score-1]

            # Adjust text color based on background color
            color = 'white' if score >= 3 else 'black'
            ax.text(j, i, f'{score}\n{label}',
                   ha="center", va="center", color=color,
                   fontweight='bold', fontsize=9)

    # Color bar
    cbar = plt.colorbar(im, ax=ax, shrink=0.6)
    cbar.set_label('Performance Score', rotation=270, labelpad=20)
    cbar.set_ticks([1, 2, 3, 4, 5])
    cbar.set_ticklabels(['1 (Poor)', '2 (Low)', '3 (Average)', '4 (Good)', '5 (Excellent)'])

    # Title
    plt.title('Key-Value Store Design Patterns: Performance Characteristics Heatmap',
             fontsize=16, fontweight='bold', pad=20)

    plt.tight_layout()
    return fig

def create_detailed_charts():
    """Creates a set of detailed charts."""
    # Create figure with 6 subplots, but the last one will be polar
    fig = plt.figure(figsize=(18, 12))
    fig.suptitle('Key-Value Store Performance Detailed Analysis', fontsize=16, fontweight='bold')

    # Create regular subplots for first 5 charts
    ax1 = plt.subplot(2, 3, 1)
    ax2 = plt.subplot(2, 3, 2)
    ax3 = plt.subplot(2, 3, 3)
    ax4 = plt.subplot(2, 3, 4)
    ax5 = plt.subplot(2, 3, 5)

    # Create polar subplot for radar chart
    ax6 = plt.subplot(2, 3, 6, projection='polar')

    # Real data
    qps_light = [300, 168, 230, 195, 126, 250, 265, 283, 243, 256]
    qps_medium = [248, 192, 212, 236, 124, 253, 268, 249, 254, 256]
    qps_heavy = [247, 174, 207, 222, 132, 0.89, 261, 283, 250, 245]
    response_time = [3.3, 5.9, 4.3, 4.3, 7.9, 3.9, 3.7, 3.5, 4.1, 3.9]
    cpu_usage = [0.05, 0.05, 0.05, 0.22, 0.03, 0.05, 0.04, 0.05, 0.04, 0.04]

    # 1. QPS Comparison (Light Load)
    bars1 = ax1.bar(range(len(patterns)), qps_light, color='skyblue', alpha=0.7)
    ax1.set_title('QPS Comparison (Light Load)')
    ax1.set_ylabel('QPS')
    ax1.set_xticks(range(len(patterns)))
    ax1.set_xticklabels([p.split('-')[0] for p in patterns], rotation=45, ha="right")

    # Highlight the maximum value
    max_idx = qps_light.index(max(qps_light))
    bars1[max_idx].set_color('orange')

    # 2. QPS Trend (By Load)
    x = np.arange(len(patterns))
    width = 0.25
    ax2.bar(x - width, qps_light, width, label='Light Load', alpha=0.7)
    ax2.bar(x, qps_medium, width, label='Medium Load', alpha=0.7)
    ax2.bar(x + width, qps_heavy, width, label='Heavy Load', alpha=0.7)
    ax2.set_title('QPS Comparison by Load')
    ax2.set_ylabel('QPS')
    ax2.set_xticks(x)
    ax2.set_xticklabels([p.split('-')[0] for p in patterns], rotation=45, ha="right")
    ax2.legend()

    # 3. Response Time
    bars3 = ax3.bar(range(len(patterns)), response_time, color='lightcoral', alpha=0.7)
    ax3.set_title('Average Response Time')
    ax3.set_ylabel('Time (ms)')
    ax3.set_xticks(range(len(patterns)))
    ax3.set_xticklabels([p.split('-')[0] for p in patterns], rotation=45, ha="right")

    # Highlight the minimum value (fastest)
    min_idx = response_time.index(min(response_time))
    bars3[min_idx].set_color('green')

    # 4. CPU Usage
    bars4 = ax4.bar(range(len(patterns)), cpu_usage, color='lightgreen', alpha=0.7)
    ax4.set_title('CPU Usage')
    ax4.set_ylabel('Usage (%)')
    ax4.set_xticks(range(len(patterns)))
    ax4.set_xticklabels([p.split('-')[0] for p in patterns], rotation=45, ha="right")

    # Highlight the outlier
    max_cpu_idx = cpu_usage.index(max(cpu_usage))
    bars4[max_cpu_idx].set_color('red')

    # 5. Scalability (QPS Change Rate)
    scalability = []
    for i in range(len(patterns)):
        if qps_light[i] > 0:
            change = (qps_heavy[i] - qps_light[i]) / qps_light[i] * 100
            scalability.append(change)
        else:
            scalability.append(-100)

    colors = ['green' if x >= -5 else 'orange' if x >= -20 else 'red' for x in scalability]
    ax5.bar(range(len(patterns)), scalability, color=colors, alpha=0.7)
    ax5.set_title('Scalability (QPS Change Rate)')
    ax5.set_ylabel('Change Rate (%)')
    ax5.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    ax5.set_xticks(range(len(patterns)))
    ax5.set_xticklabels([p.split('-')[0] for p in patterns], rotation=45, ha="right")

    # 6. Overall Rating Radar Chart
    # Compare top 3 patterns
    top_patterns = ['rate-limiting', 'cache-aside', 'line-streams']
    top_indices = [patterns.index(p) for p in top_patterns]

    angles = np.linspace(0, 2*np.pi, len(metrics)-1, endpoint=False).tolist()
    angles += angles[:1]  # Close the circle

    ax6.set_theta_offset(np.pi / 2)
    ax6.set_theta_direction(-1)
    ax6.set_thetagrids(np.degrees(angles[:-1]), metrics[:-1])

    colors_radar = ['red', 'blue', 'green']
    for i, idx in enumerate(top_indices):
        values = scores[idx][:-1].tolist()  # Exclude overall rating
        values += values[:1]  # Close the circle
        ax6.plot(angles, values, 'o-', linewidth=2,
                label=patterns[idx], color=colors_radar[i])
        ax6.fill(angles, values, alpha=0.25, color=colors_radar[i])

    ax6.set_ylim(0, 5)
    ax6.set_title('Top 3 Patterns Comparison')
    ax6.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

    plt.tight_layout()
    return fig

def create_ranking_chart():
    """Creates the ranking chart."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), gridspec_kw={'width_ratios': [3, 4]})

    # Calculate total scores
    total_scores = np.sum(scores, axis=1)
    sorted_indices = np.argsort(total_scores)[::-1]

    # 1. Overall Ranking
    ranking_colors = ['gold', 'silver', '#cd7f32', 'lightblue', 'lightgreen',
                     'lightyellow', 'lightcoral', 'lightgray', 'lightpink', 'lightcyan']

    bars1 = ax1.barh(range(len(patterns)),
                     [total_scores[i] for i in sorted_indices],
                     color=[ranking_colors[i] for i in range(len(patterns))])

    ax1.set_yticks(range(len(patterns)))
    ax1.set_yticklabels([pattern_labels[i] for i in sorted_indices])
    ax1.invert_yaxis() # Highest score on top
    ax1.set_xlabel('Total Score')
    ax1.set_title('Overall Performance Ranking')

    # Display score on the bar
    for i, (idx, score) in enumerate(zip(sorted_indices, [total_scores[i] for i in sorted_indices])):
        ax1.text(score + 0.5, i, f'{score} pts', va='center', fontweight='bold')

    # 2. Recommendation by Use Case
    use_cases = ['Web App', 'API Gateway', 'Gaming', 'Finance', 'Distributed DB']
    recommendations = {
        'Web App': [4, 2, 4, 2, 5, 1, 4, 3, 5, 3],      # Highlighting session-store, cache-aside
        'API Gateway': [3, 2, 3, 2, 3, 1, 5, 4, 3, 2],     # Highlighting rate-limiting
        'Gaming': [3, 2, 4, 3, 3, 2, 3, 4, 3, 5],          # Highlighting leaderboard, line-streams
        'Finance': [4, 5, 4, 3, 3, 2, 3, 3, 3, 3],            # Highlighting quorum-consistency
        'Distributed DB': [5, 4, 5, 4, 3, 3, 3, 4, 4, 4]   # Highlighting coordinator-ring, sharding
    }

    # Display as a heatmap
    rec_matrix = np.array([recommendations[uc] for uc in use_cases])
    im2 = ax2.imshow(rec_matrix, cmap='RdYlGn', aspect='auto', vmin=1, vmax=5)

    ax2.set_xticks(range(len(patterns)))
    ax2.set_yticks(range(len(use_cases)))
    ax2.set_xticklabels([p.split('-')[0] for p in patterns], rotation=45, ha="right")
    ax2.set_yticklabels(use_cases)
    ax2.set_title('Recommendation Matrix by Use Case')

    # Display values
    for i in range(len(use_cases)):
        for j in range(len(patterns)):
            ax2.text(j, i, f'{rec_matrix[i, j]}',
                    ha="center", va="center", fontweight='bold')

    plt.colorbar(im2, ax=ax2, shrink=0.8, label='Recommendation Score')
    plt.tight_layout()
    return fig

def main():
    """Main execution function."""
    print("Creating Key-Value store design pattern heatmaps...")

    # 1. Main Heatmap
    fig1 = create_heatmap()
    fig1.savefig('kvstore_heatmap_en.png', dpi=300, bbox_inches='tight')

    # 2. Detailed Analysis Charts
    fig2 = create_detailed_charts()
    fig2.savefig('kvstore_detailed_analysis_en.png', dpi=300, bbox_inches='tight')

    # 3. Ranking Chart
    fig3 = create_ranking_chart()
    fig3.savefig('kvstore_ranking_en.png', dpi=300, bbox_inches='tight')

    # Show plots
    plt.show()

    print("\n✅ Heatmaps created successfully.")
    print("📊 Generated Files:")
    print("  - kvstore_heatmap_en.png: Main Heatmap")
    print("  - kvstore_detailed_analysis_en.png: Detailed Analysis")
    print("  - kvstore_ranking_en.png: Ranking & Recommendations")

if __name__ == "__main__":
    main()